# カテゴリデータをワンホット表現にする話

In [1]:
import pandas as pd
import sklearn.preprocessing as sp

In [2]:
data = pd.read_csv("menu.csv", skipinitialspace=True)

In [3]:
data

,ID,MENU,MATERIAL
0,0,肉じゃが,じゃがいも
1,1,きんぴら,にんじん
2,2,オニオンリング,たまねぎ
3,3,ポテトサラダ,じゃがいも
4,4,もみじおろし,にんじん
5,5,フライドポテト,じゃがいも


##  ラベルエンコードを実施する

In [4]:
le = sp.LabelEncoder()
le.fit(data.MATERIAL.unique())
data.MATERIAL = le.fit_transform(data.MATERIAL)
data

,ID,MENU,MATERIAL
0,0,肉じゃが,0
1,1,きんぴら,2
2,2,オニオンリング,1
3,3,ポテトサラダ,0
4,4,もみじおろし,2
5,5,フライドポテト,0


In [5]:
#  一旦もとにもどす
data.MATERIAL = le.inverse_transform(data.MATERIAL)
data

,ID,MENU,MATERIAL
0,0,肉じゃが,じゃがいも
1,1,きんぴら,にんじん
2,2,オニオンリング,たまねぎ
3,3,ポテトサラダ,じゃがいも
4,4,もみじおろし,にんじん
5,5,フライドポテト,じゃがいも


In [6]:
data.MATERIAL = le.transform(data.MATERIAL) # ラベルエンコードし直す
ohe = sp.OneHotEncoder()
enced = ohe.fit_transform(data.MATERIAL.values.reshape(1, -1).transpose())

In [7]:
temp = pd.DataFrame(index=data.MATERIAL.index, columns="MATERIAL-" + le.classes_, data=enced.toarray())
temp

,MATERIAL-じゃがいも,MATERIAL-たまねぎ,MATERIAL-にんじん
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
5,1.0,0.0,0.0


In [8]:
enced_data = pd.concat([data, temp], axis=1)
del enced_data['MATERIAL']
enced_data

,ID,MENU,MATERIAL-じゃがいも,MATERIAL-たまねぎ,MATERIAL-にんじん
0,0,肉じゃが,1.0,0.0,0.0
1,1,きんぴら,0.0,0.0,1.0
2,2,オニオンリング,0.0,1.0,0.0
3,3,ポテトサラダ,1.0,0.0,0.0
4,4,もみじおろし,0.0,0.0,1.0
5,5,フライドポテト,1.0,0.0,0.0


In [9]:
# Testデータにも反映する時
ohe.transform(data.MATERIAL.values.reshape(1, -1).transpose()).toarray()

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.]])